In [10]:
import search
reload(search)

# reopen index, but reload it instead of creating it afresh
udacity_se = search.UdacitySearchEngine(create=False)
    
# dictionary of results
results = udacity_se.search("apple")
print results

[{'slug': u'grand-central-dispatch-gcd--ud576', 'title': u'Grand Central Dispatch (GCD)'}, {'slug': u'intro-to-ios-app-development-with-swift--ud585', 'title': u'Intro to iOS App Development with Swift'}, {'slug': u'swift-for-beginners--ud1022', 'title': u'Swift for Beginners'}, {'slug': u'server-side-swift--ud1031', 'title': u'Server-Side Swift'}, {'slug': u'learn-swift-programming-syntax--ud902', 'title': u'Learn Swift Programming Syntax'}, {'slug': u'ios-persistence-and-core-data--ud325', 'title': u'iOS Persistence and Core Data'}]


In [21]:
import search
reload(search)

# reopen index, but reload it instead of creating it afresh
se = search.HarvardXSearchEngine(create=False)
    
# dictionary of results
results = se.search("Psychology")
print results

[{'course_id': u'course-v1:HarvardX+GSE4x+2T2016', 'display_name': u'Kathleen Hoover-Dempsey Bio'}, {'course_id': u'course-v1:HarvardX+GSE4x+2T2016', 'display_name': u'Additional Resources'}, {'course_id': u'course-v1:HarvardX+GSE4x+2T2016', 'display_name': u'Watch: Parental Motivation for Engagement'}, {'course_id': u'course-v1:HarvardX+GSE4x+2T2016', 'display_name': u'Additional Resources'}, {'course_id': u'course-v1:HarvardX+PH201x+2013_SOND', 'display_name': u'Meet the Teaching Assistants: Felicia Browne'}, {'course_id': u'course-v1:HarvardX+GSE2x+1T2016', 'display_name': u'Additional Resources'}, {'course_id': u'course-v1:HarvardX+GSE2x+1T2016', 'display_name': u'Intro to Neuroeducation'}, {'course_id': u'course-v1:HarvardX+1368.2x+2T2016', 'display_name': u'Glossary'}, {'course_id': u'course-v1:HarvardX+GSE4x+2T2016', 'display_name': u'Explore the Research'}, {'course_id': u'course-v1:HarvardX+AmPoX.6+2T2016', 'display_name': u''}]


In [4]:
import gensim.models as models
import gensim.models.word2vec as word2vec

# TODO remove ngrams lol
# prepare corpus and ngram reader
# create
# corpus = word2vec.Text8Corpus('datasets/enwik8')
# ngram_phrases= models.Phrases(corpus)
# ngram = models.phrases.Phraser(ngram_phrases)
# ngram.save('models/phraser/phraser')
# or just load from file
ngram = models.phrases.Phraser.load('models/phraser/phraser')

# load model
# either or
# model = word2vec.Word2Vec(ngram[corpus], workers=8)
# model.save('models/word2vec/word2vec')
model = word2vec.Word2Vec.load('models/word2vec/word2vec')

# test model
# BUG: this doesn't work that well :(
# it worked pretty well before i put in ngrams
print model.most_similar(positive=['autism'], topn=10)

[(u'[[Vishnu]]', 0.4175434112548828), (u'rulers]]', 0.4047089219093323), (u'Hillary', 0.3996689021587372), (u'International_Conference', 0.39594656229019165), (u'[[Children', 0.3944295048713684), (u'understandable', 0.3920005261898041), (u'[[Harley', 0.38948845863342285), (u'Protocol&lt;br&gt;', 0.3832295536994934), (u'Lovecraft:', 0.3831127882003784), (u'align=left', 0.38207781314849854)]


In [22]:
import rewriter
    
test_terms = [
    'confirmation bias',
    'Albert Einstein',
    'biochemistry',
]

rw = rewriter.WikipediaRewriter()

for term in test_terms:
    print rw.rewrite(term)

['CS1 maint: Unfit url', 'Cognitive biases', 'Cognitive inertia', 'Critical thinking', 'Design of experiments', 'Error', 'Featured articles', 'Ignorance', 'Inductive fallacies', 'Logical fallacies']
['1879 births', '1955 deaths', '20th-century American engineers', '20th-century American writers', '20th-century German writers', '20th-century Jews', '20th-century physicists', 'AC with 19 elements', 'Activists from New Jersey', 'Albert Einstein']
['Biochemistry', 'Biotechnology', 'CS1 maint: Uses editors parameter', 'Molecular biology', 'Pages using ISBN magic links', 'Wikipedia articles with GND identifiers', 'Wikipedia articles with LCCN identifiers']


In [33]:
# compare query rewriting to the normal results 
import rewriter
reload(rewriter)


searches = [
    "neural networks",
    "swift",
    "venture capital",
    "object-oriented programming",
    "deep learning",
    "macOS"
]

wikidacity = rewriter.RewritingSearchEngine(rw, udacity_se)

for query in searches:
    # normal results
    num_plain_results = len(udacity_se.search(query))
    
    # with wikipedia rewriting
    num_wiki_results = len(wikidacity.search(query))
    
    print (query, num_plain_results, num_wiki_results)

[]
('neural networks', 2, 0)
['All articles with unsourced statements', 'Apodidae', "Articles with 'species' microformats", 'Articles with unsourced statements from September 2016', 'Birds by common name', 'Extant Eocene first appearances', 'Pages containing links to subscription-only content', 'Swifts']
('swift', 9, 0)
['All articles needing additional references', 'All articles with unsourced statements', 'Articles needing additional references from March 2014', 'Articles with specifically marked weasel-worded phrases from May 2014', 'Articles with unsourced statements from November 2010', 'CS1 Spanish-language sources (es)', 'Corporate finance', 'Entrepreneurship', 'Equity securities', 'Private equity']
('venture capital', 1, 0)
['All articles containing potentially dated statements', 'All articles needing additional references', 'All articles with failed verification', 'All articles with unsourced statements', 'Articles containing potentially dated statements from 2006', 'Articles 